# Generador de Datos Sintéticos con LLaMA-2

Este script implementa un **generador de datos sintéticos** basado en prompts generados también de forma **sintética**, con el objetivo de **incrementar la diversidad** de los textos generados.

 **Requisitos importantes**:

1. **Token de Hugging Face**: Es necesario tener un token de acceso válido a Hugging Face.  
2. **Aceptación de Términos y Condiciones**: Debes aceptar los términos de uso de los modelos de **Meta-LLaMA** para poder descargar y usar los checkpoints.

---

## Configuración del token

Crea un archivo llamado `secrets.json` en el mismo directorio del proyecto con la siguiente estructura:

```json
{
    "huggingface_token": "TU_TOKEN_AQUI"
}


In [6]:
!git clone https://github.com/bitsandbytes-foundation/bitsandbytes.git
%cd bitsandbytes
!pip install cmake
!cmake -DCOMPUTE_BACKEND=cuda -S . -B build
!cmake --build build --config Release
!pip install .


fatal: destination path 'bitsandbytes' already exists and is not an empty directory.
/kaggle/working/bitsandbytes/bitsandbytes
CMake Error: The source directory "/kaggle/working/bitsandbytes/bitsandbytes" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
Error: /kaggle/working/bitsandbytes/bitsandbytes/build is not a directory
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [3]:
!pip install transformers torch
# !pip install -U bitsandbytes --no-deps

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [4]:
import os
import json
import textwrap
import bitsandbytes 
from typing import Tuple, Optional
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = "cuda" if torch.cuda.is_available() else "cpu"

# with open("secrets.json", "r", encoding="utf-8") as f:
#     secrets = json.load(f)
#HF_TOKEN = secrets["huggingface_token"]

#HF_TOKEN =  en entorno kagle escribir aquí 
# Prompt builder
def random_applicant() -> dict:
    """Genera un perfil de solicitante aleatorio (en inglés, con más variedad y contexto)."""
    import random

    # --- Datos básicos ---
    first_names = [
        "Alex", "Jamie", "Taylor", "Jordan", "Morgan", "Riley", "Casey", "Chris", "Avery", "Drew",
        "Samantha", "Daniel", "Sophia", "Michael", "Olivia", "Liam", "Isabella", "Noah", "Ethan", "Emma"
    ]
    last_names = [
        "Johnson", "Smith", "Lee", "Patel", "Williams", "Garcia", "Brown", "Davis", "Miller", "Wilson",
        "Martinez", "Anderson", "Clark", "Lopez", "Lewis", "Walker", "Young", "Allen", "King"
    ]

    # --- Roles profesionales ---
    roles = [
        "Data Scientist", "Software Engineer", "Frontend Developer", "UX Designer",
        "Marketing Manager", "Business Analyst", "Product Manager", "Research Scientist",
        "Sales Executive", "Customer Success Manager", "Financial Analyst", "HR Coordinator",
        "Operations Lead", "IT Support Specialist", "DevOps Engineer", "Content Strategist",
        "Mechanical Engineer", "Project Manager", "Copywriter", "Legal Assistant"
    ]

    # --- Empresas ficticias ---
    companies = [
        "Acme Corp", "TechNova", "BlueSky Labs", "FutureWorks", "Quantum Analytics", "NextGen Solutions",
        "EverBright", "Pioneer Systems", "OrbitSoft", "NeuralEdge", "BrightLeaf Consulting", "SkyBridge AI",
        "Helix Dynamics", "CleverPath", "OptiData", "Sunrise Media", "GreenFlow Technologies"
    ]

    # --- Sectores o industrias ---
    industries = [
        "technology", "finance", "education", "healthcare", "energy", "marketing", "consulting",
        "manufacturing", "media", "logistics", "environmental science", "AI research", "non-profit"
    ]

    # --- Nivel educativo / académico ---
    education_levels = [
        "Bachelor's in Computer Science", "Master's in Data Analytics", "MBA",
        "Bachelor's in Marketing", "PhD in Artificial Intelligence", "Bachelor's in Business Administration",
        "Master's in Mechanical Engineering", "Bachelor's in Graphic Design", "BSc in Economics",
        "Bachelor's in Psychology", "BA in Communications"
    ]

    # --- Tonos posibles ---
    tones = [
        "professional", "enthusiastic", "confident", "humble but ambitious", "creative", "friendly", "formal",
        "inspirational", "passionate about innovation", "results-driven", "empathetic", "analytical"
    ]

    # --- Habilidades ---
    skills_pool = [
        "Python", "R", "Java", "C++", "SQL", "Machine Learning", "Deep Learning", "Project Management",
        "Marketing Strategy", "Data Visualization", "Cloud Computing", "Communication", "Leadership",
        "Team Collaboration", "Research", "Excel", "Power BI", "TensorFlow", "React", "Docker", "Kubernetes",
        "SEO Optimization", "Public Speaking", "Customer Relations", "Financial Modeling"
    ]

    # --- Extras que afectan el prompt ---
    extras = [
        "Emphasize measurable impact and leadership.",
        "Highlight adaptability and eagerness to learn new tools.",
        "Show strong cross-functional collaboration and communication.",
        "Focus on creative problem-solving and curiosity for innovation.",
        "Mention the applicant’s commitment to diversity and inclusion.",
        "Emphasize attention to detail and efficiency.",
        "Showcase ability to work under pressure and meet deadlines.",
        "Highlight mentorship and team motivation skills.",
        "Include a brief anecdote that reflects passion for the field.",
        "Keep the tone optimistic and forward-thinking."
    ]

    # --- Generación aleatoria ---
    name = f"{random.choice(first_names)} {random.choice(last_names)}"
    role = random.choice(roles)
    company = random.choice(companies)
    tone = random.choice(tones)
    industry = random.choice(industries)
    education_level = random.choice(education_levels)
    extra = random.choice(extras)

    n_skills = random.randint(4, 7)
    skills_list = random.sample(skills_pool, n_skills)

    exp_years = random.randint(1, 15)
    exp_focus = random.choice([
        f"developing {role.lower()} solutions for the {industry} sector",
        f"driving innovation and data-driven decisions",
        f"leading multidisciplinary teams in fast-paced environments",
        f"managing end-to-end projects and achieving KPIs",
        f"improving processes and optimizing efficiency"
    ])

    experience_summary = f"{exp_years} years of experience {exp_focus}."

    return {
        "name": name,
        "role": role,
        "company": company,
        "industry": industry,
        "education_level": education_level,
        "experience_summary": experience_summary,
        "skills_list": skills_list,
        "tone": tone,
        "extra": extra
    }


def build_cover_letter_prompt(applicant: dict) -> str:
    """Construye prompt consistente para generar la cover letter (en inglés)."""
    skills = ", ".join(applicant.get("skills_list", [])) or "N/A"
    extra = applicant.get("extra", "")
    prompt = f"""
You are an expert career advisor and professional cover letter writer.

Write a concise, persuasive English cover letter (job application / cover letter) using the information below.
Return only the letter body (no metadata or commentary).

Applicant:
- Name: {applicant.get('name', 'Applicant')}
- Target role: {applicant.get('role', '')}
- Target company: {applicant.get('company', '')}
- Short experience summary: {applicant.get('experience_summary', '')}
- Key skills: {skills}
- Tone: {applicant.get('tone', 'professional')}
- Extra: {extra}

Requirements:
- Length: 150-300 words.
- Include a tailored opening line mentioning the company and role.
- Include 2-3 sentences highlighting accomplishments/results (use approximate numbers if necessary).
- Use a clear closing paragraph inviting next steps and a polite sign-off.
- Do not include real contact details; use placeholders if needed.

Begin the letter with an appropriate greeting (e.g., "Dear Hiring Manager," or "Dear <Company> Recruiting Team,").
"""
    return textwrap.dedent(prompt).strip()

bitsandbytes library load error: Configured CUDA binary not found at /kaggle/working/bitsandbytes/bitsandbytes/libbitsandbytes_cuda124.so
Traceback (most recent call last):
  File "/kaggle/working/bitsandbytes/bitsandbytes/cextension.py", line 320, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/kaggle/working/bitsandbytes/bitsandbytes/cextension.py", line 288, in get_native_library
    raise RuntimeError(f"Configured {BNB_BACKEND} binary not found at {cuda_binary_path}")
RuntimeError: Configured CUDA binary not found at /kaggle/working/bitsandbytes/bitsandbytes/libbitsandbytes_cuda124.so


In [5]:
# Ajustes: cambia el nombre del modelo según lo que tengas disponible localmente
# Para Llama2 en HF: "meta-llama/Llama-2-7b-chat-hf"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

load_in_4bit = 0

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

# Opción A: intentar cargar en 4-bit (recomendado si bitsandbytes soporta tu entorno)
if load_in_4bit:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype="auto",
        use_auth_token=HF_TOKEN
    )
    print("cargado 4bits")
else:
    # Opción B: cargar en 8-bit
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        load_in_8bit=True,
        torch_dtype="auto",
        token=HF_TOKEN
    )
    print("cargado 8bits")
    
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,use_auth_token=HF_TOKEN)

# asegúrate de que tokenizer.eos_token_id existe
if tokenizer.eos_token_id is None:
    tokenizer.eos_token_id = tokenizer.pad_token_id or 0

#base
gen_kwargs = {
        "max_new_tokens": 300,
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.95,
        "repetition_penalty": 1.05,
        "pad_token_id": tokenizer.eos_token_id}

PackageNotFoundError: No package metadata was found for bitsandbytes

In [ ]:
def generate(prompt, model=model, tokenizer=tokenizer, gen_kwargs=gen_kwargs):
    """
    model: modelo cargado
    tokenizer: tokenizer
    prompt: string
    gen_kwargs: dict con argumentos para model.generate()
    """

    # tokenizar prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # generar
    out = model.generate(**inputs, **gen_kwargs)

    # extraer solo lo generado
    start_len = inputs["input_ids"].shape[1]
    generated_ids = out[0][start_len:]
    text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return text


In [ ]:
def generate_n_cover_letters(n: int, model, tokenizer, output_file="cover_letters.jsonl"):

    with open(output_file, "w", encoding="utf-8") as f:
        for i in range(n):
            applicant = random_applicant()
            prompt = build_cover_letter_prompt(applicant)

            letter = generate(
                model=model,
                tokenizer=tokenizer,
                prompt=prompt,
                gen_kwargs=gen_kwargs
            )

            # guardar en JSONL
            record = {
                "applicant": applicant,
                "cover_letter": letter
            }

            f.write(json.dumps(record, ensure_ascii=False) + "\n")
            print(f"[+] Generada carta {i+1}/{n}")


### abajo basura

In [ ]:
#  Ejemplo de uso 

sample_applicant = {
    "name": "Alex Johnson",
    "role": "Data Scientist",
    "company": "Acme Tech",
    "experience_summary": "3 years building production ML pipelines and improving prediction accuracy for customer churn.",
    "skills_list": ["Python", "pandas", "scikit-learn", "SQL", "model deployment"],
    "tone": "professional and confident",
    "extra": "Emphasize cross-team collaboration and measurable impact."
}

# Genera usando LLaMA local (intenta 8-bit)
try:
    llm_cover = generate_cover_llama_local(sample_applicant, use_8bit=True, temperature=0.7)
    print("\n--- LLaMA COVER ---\n", llm_cover[:2000])
except Exception as e:
    LOG.error("LLaMA generation failed: %s", e)

# Genera usando HF model local (p. ej. Falcon)
try:
    hf_cover = generate_cover_hf_local(sample_applicant, use_8bit=True, temperature=0.9)
    print("\n--- HF COVER ---\n", hf_cover[:2000])
except Exception as e:
    LOG.error("HF generation failed: %s", e)


In [ ]:
import pandas as pd
from tqdm import tqdm

def generate_cover_dataset(
    n: int = 10,
    model_type: str = "llama",  # o "hf"
    temperature: float = 0.7,
    use_8bit: bool = True,
    use_4bit: bool = False,
    secrets_path: str = "secrets.json",
    save_path: Optional[str] = None,
) -> pd.DataFrame:
    """
    Genera un dataset de cover letters con prompts y resultados.
    model_type: "llama" o "hf"
    """
    data = []

    LOG.info(f"Generating {n} cover letters using model_type={model_type}...")

    for _ in tqdm(range(n), desc="Generating covers"):
        applicant = random_applicant()
        prompt = build_cover_letter_prompt(applicant)

        try:
            if model_type.lower() == "llama":
                letter = generate_cover_llama_local(
                    applicant,
                    secrets_path=secrets_path,
                    use_8bit=use_8bit,
                    use_4bit=use_4bit,
                    temperature=temperature,
                )
            else:
                letter = generate_cover_hf_local(
                    applicant,
                    secrets_path=secrets_path,
                    use_8bit=use_8bit,
                    use_4bit=use_4bit,
                    temperature=temperature,
                )
        except Exception as e:
            LOG.error("Error generating letter: %s", e)
            letter = f"[ERROR] {e}"

        row = {
            **applicant,
            "prompt": prompt,
            "cover_letter": letter,
        }
        data.append(row)

    df = pd.DataFrame(data)
    LOG.info(" Dataset generated with %d samples", len(df))

    if save_path:
        df.to_csv(save_path, index=False)
        LOG.info("Dataset saved to %s", save_path)

    return df
